<h1 style="text-align: center;"><b>Home comfort and energy</b></h1>

<img 
    style="display: block; 
           margin-left: auto;
           margin-right: auto;
           width: 8%;"
    src="https://www.ec-lyon.fr/sites/default/files/styles/full/public/legacy-files/logo_ecl_carre_q.png?itok=gG2EmtXL" 
    alt="Our logo">
</img>

<h2 style="text-align: center;"><b>BE - Report</b></h2>

<h4 style="text-align: center;">27/02/2023</h4>

<p style="text-align: left;">Students<br>Boris LAURENT<br>Salomé RICHARD<br>Léo PINCHARD<br>Marie d'HARCOURT</p>

<p style="text-align: right;">Teacher<br>Christian GHIAUS</p>






# Introduction
The building sector represents 44% of the energy consumed in France, far ahead of the transport sector. Each year, the building sector emits more than 123 million tons of C02, which makes it one of the key areas in the fight against global warming and the energy transition.<br>
In this BE :
- We will discuss the influence of different parameters in the behavior of a simple building, based on a code previously developed.
We will study four cases :

Case 1. The four rooms are controlled by a HVAC system at the setpoint temperatures 20 °C, 20 °C, 22 °C, 18 °C for rooms 1, 2, 3, and 4, respectively and that the there is no solar radiation on the walls. 

Case 2. We consider that there is solar radiation absorbed by the walls and that rooms 1 and 3 are controlled by a HVAC system while rooms 2 and 4 are in "free-running". 

Case 3. Room 2 is ventilated with an air-flow rate of $ACH = 1 \, \mathrm{volume /hour}$ (ACH is the number of air-changes per hour). The density and specific heat of the air are $\rho = 1.2 \, \mathrm{kg/m^3}$ and $c = 1000 \, \mathrm{J/(kg \, K)}$, respectively). The outdoor air enters in room 2, then is moves to room 4 and, from here, it gets outdoors.

Case 4. The air-flow in room 2 is reversed: from outdoors, the air enters in room 4, then it moves to room 2, from which it gets outdoors. Considering that the air flow rate of room 2 is the same as before (i.e., $ACH = 1 \, \mathrm{volume /hour}$).

The objective is to discuss the influence of different parameters, such as ventilation, materials and solar radiation on the thermal loads and the temperatures of the indoor air of the rooms.

- Then, we will focus on the modeling of a more complex architecture, resulting from the TET option project "Ferme du Milieu à Lhuis". 


# First part - four room's building - Description of the problem
## Problem
### Data
The building is formed of 4 rooms with the dimensions (Figure 1):  
- $L = 6, \, l = 3\,  \mathrm{m}$, lengths,
- $H = 6 \, \mathrm{m}$, height.
- $w = 0.20 \, \mathrm{m}$, width of the walls.

![fig](./figures/08_bldg_plan.svg)
> Figure 1. Plan of the building

The thermal conductivity of the walls is $\lambda = 1.7 \, \mathrm{W/(m \, K)}$.

The convective heat coefficients between the walls and the indoor and outdoor air are $h_i = 8 \, \mathrm{W/(m^2 \, K)}$ and $h_o = 25 \, \mathrm{W/(m^2 \, K)}$ respectively.

In **winter** conditions, the short-wave solar radiation absorbed by each wall is $E = 200 \, \mathrm{W/m^2}$ and the outdoor temperature is $T_o = 0 \, \mathrm{°C}$.

In **summer** conditions, the short-wave solar radiation absorbed by each wall is $E = 400 \, \mathrm{W/m^2}$ and the outdoor temperature is $T_o = 30 \, \mathrm{°C}$.


In [1]:
import numpy as np

np.set_printoptions(precision=1)

# Data
# ====
# dimensions
L, l, H, w = 6, 3, 3, 0.20  # m

# thermo-physical propertites
λ = 1.7             # W/(m K) wall thermal conductivity
ρ, c = 1.2, 1000    # kg/m3, J/(kg K) density, specific heat air
hi, ho = 8, 25      # W/(m2 K) convection coefficients in, out

# short-wave solar radiation absorbed by each wall
E = 200             # W/m2

# outdoor temperature
To = 0              # °C

# ventilation rate (air-changes per hour)
ACH = 1             # volume/h

## Thermal model

The [volumetric flow rate](https://en.m.wikipedia.org/wiki/Volumetric_flow_rate) of air circulating in room 2 results from the ventilation rate:
$$\dot V = L·l·H·\frac{ACH}{3600 \, \mathrm{s}}$$

The [mass flow rate](https://en.m.wikipedia.org/wiki/Mass_flow_rate) of air entering and getting out of room 2 is:
$$\dot m = \rho \dot V$$

In [2]:
V_dot = L * l * H * ACH / 3600  # volumetric air flow rate
m_dot = ρ * V_dot               # mass air flow rate

### Thermal conductances

In the model, we will consider the conductances:
- $h \, S$ - convection, W/K;
- $\lambda / w \, S$ - conduction, W/K;
- $\dot m c$ - advection, W/K;
- $K_p$ - gain of the proportional controller, W/K.

**Note:** in determening the surfce area of the walls $S = L \, l \, H$ the length of the walls are the indoor values.

We will consider the conductances of exterior and the interior walls (Figure 2). The exterior walls are modelled with a conductance for outdoor convection (cyan in Figure 2) and an equivalent conductance for conduction in the wall and indoor convection (blue in Figure 2). The indoor walls are modelled by an equivalent conductance for indoor convection, conduction, and indoor convection on the other side.

![fig](./figures/08_blds_conductance.svg)
> Figure 2. Walls forming thermal coductances in the model.

### Thermal network

The thermal network modeling the building is a [weighted directed graph](https://en.m.wikipedia.org/wiki/Directed_graph) (Figure 3). It is composed of edges (or branches) and vertexes (or nodes).

The temperature nodes model a volume, a surface, a line or a point in space for which the temperature is considered homogenous. The nodes are dependent temperatures (outputs of the physical system). The reference temperature is considered zero Celsius; it has a known value and is represented by a bar (Figure 3).

To a temperature node $\theta_j$, there are connected [heat flow rate](https://en.m.wikipedia.org/wiki/Rate_of_heat_flow) branches $q_k, k = 0 ... n$ and [heat flow rate sources](https://en.m.wikipedia.org/wiki/Current_source), $f_j$ (Figure 3a). A heat rate source delivers a heat flow rate whatever is the temperature difference. In buildings, typical heat flow rate sources are [solar irradiance](https://en.m.wikipedia.org/wiki/Solar_irradiance), [electrical appliances](https://en.wikipedia.org/wiki/Joule_heating) and [occupants](https://en.wikipedia.org/wiki/Thermal_comfort#Metabolic_rate). If a heat flow rate source is [identically zero](https://en.wiktionary.org/wiki/identically_zero), then it is not figured on the graph.

In steady-state, the [algebraic sum of the heat flow rates](https://en.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws#Kirchhoff's_current_law) entering the temperature node $\theta_j$ is zero:
$$\sum_{k} q_k + f_j= 0$$

Two temperature nodes $\theta_i$ and $\theta_j$ are connected by directed edges of heat flow rates $q_k$ (Figure 3a). On each edge, there is a conductance $G_k$ and a temperature source $b_k$. A [temperature source](https://en.m.wikipedia.org/wiki/Voltage_source) maintains a temperature value whatever is the heat flow rate. In building, outdoor air, ground, and setpoint temperatures are typical examples of temperature sources. If a temperature source is [identically zero](https://en.wiktionary.org/wiki/identically_zero), then it is not figured on the graph.

From the [directed sum of temperature differences over a branch](https://en.m.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws#Kirchhoff's_voltage_law), it results that the [temperature drop](https://en.wikipedia.org/wiki/Voltage_drop) over a conductance is (Figure 3a):
$$e_k = \theta_i - \theta_j + b_k$$

The relation between the temperature drop, $e_k$, over a conductance, $G_k$, and the heat flow rate, $q_k$, thrrough the conductance $G_k$ is:
$$q_k = G_k \, e_k$$

This [weighted directed graph](https://en.m.wikipedia.org/wiki/Directed_graph) is characterized by:
- $A$ - [incidence matrix](https://en.m.wikipedia.org/wiki/Incidence_matrix);
- $G$ - [conductance](https://en.m.wikipedia.org/wiki/Electrical_resistance_and_conductance)  diagonal matrix;
- $b$ - vector of [temperature sources](https://en.m.wikipedia.org/wiki/Voltage_source);
- $f$ - vector of [flow-rate sources](https://en.m.wikipedia.org/wiki/Current_source).


The solution of the problem is ([Ghiaus (2013)](https://hal.archives-ouvertes.fr/hal-03605823/document)):

$$ \theta = (A^TGA)^{-1}(A^TGb + f)$$
and
$$ q = G(-A\theta + b)$$

![fig](./figures/08_thermal_network.svg)
> Figure 3. Thermal network: a) typical element of the thermal network (the sources are not figured if they are identically zero); b) thermal network model for the buiding shown in Figures 1 and 2.

For our model, the graph has 20 braches and 8 nodes (Figure 3b).

In [3]:
nq, nθ = 20, 8  # number of flow-rates branches and of temperaure nodes

### Incidence matrix
The [incidence matrix of a directed graph](https://en.m.wikipedia.org/wiki/Incidence_matrix) shows the relationship between the edges (or branches) $k$ and the vertexes (or nodes) $j$ of the graph (Figure 3a):

$$A_{k,j} = \begin{cases}\phantom{-}
0 & \text{if heat flow rate } q_k \text{ is not connected to node }  \theta_j \\ 
+1 & \text{if heat flow rate } q_k \text{ enters into node }  \theta_j\\ 
-1 & \text{if heat flow rate } q_k \text{ gets out of node }  \theta_l 
\end{cases}$$

The incidence matrix is a [sparce matrix](https://en.m.wikipedia.org/wiki/Sparse_matrix) which has:
- the number of rows equal to $n_q$, the number of flow rates $q_k$;
- the number of columns equal to $n_{\theta}$, the number of temperature nodes $\theta_j$.

In [4]:
# Incidence matrix
# ================
A = np.zeros([nq, nθ])

# q0 ... q3 (cyan branches)
A[0, 0] = 1
A[1, 1] = 1
A[2, 5] = 1
A[3, 7] = 1

# q4 ... q7 (blue branches)
A[4, 0], A[4, 3] = -1, 1
A[5, 1], A[5, 2] = -1, 1
A[6, 4], A[6, 5] = -1, 1
A[7, 7], A[7, 6] = -1, 1

# q8 ... q12 (yellow branches)
A[8, 2], A[8, 3] = -1, 1
A[9, 3], A[9, 4] = -1, 1
A[10, 6], A[10, 2] = -1, 1
A[11, 6], A[11, 3] = -1, 1
A[12, 6], A[12, 4] = -1, 1

# q13 ... q15 (green branches)
A[13, 3] = 1
A[14, 3], A[14, 6] = -1, 1
A[15, 6] = 1

# q16 ... q19 (red branches)
A[16, 2] = 1
A[17, 3] = 1
A[18, 4] = 1
A[19, 6] = 1

### Conductances
The number of conductances correspod to the number of flows which is the number of rows of incidence matrix $A$

In [5]:
# Conductance matrix
# ==================
G = np.zeros(A.shape[0])

#### Exterior walls: outdoor convection
The conductances of [convection](https://en.m.wikipedia.org/wiki/Newton%27s_law_of_cooling) between the walls and the outdoor air (in cyan in Figures 2 and 3) are of the form
$$G_i = h_o \, S_{o,k},\text{ with } k = 0 ... 3$$
where $S_{o,k}$ is the surface area of the wall in contact with the outdoor air:
- $S_{o,0} = L \, H$;
- $S_{o,1} = S_{o,2} = (L + l) H$;
- $S_{o,3} = (2l + 3L + 2w)H$.


In [6]:
# G0 ... G3 (cyan branches): outdoor convection
L4 = 2 * l + 3 * L + 2 * w      # length outdoor wall room 4
So = np.array([L, L + l, L + l, L4]) * H    # outdoor surfaces
G[0:4] = ho * So

#### Exterior walls: conduction and indoor convection
The [equivalent conductance](https://en.m.wikipedia.org/wiki/Series_and_parallel_circuits#Combining_conductances) for [conduction](https://en.m.wikipedia.org/wiki/Thermal_conduction) in the wall and [convection](https://en.m.wikipedia.org/wiki/Newton%27s_law_of_cooling) between the wall and the indoor air (in blue in Figure 3) is:
$$G_k = \frac{1}{\frac{w}{\lambda} + \frac{1}{h_i}}S_{o,j},\text{ with } k = 4 ... 7, j = 0 ... 3$$

In [7]:
# G4 ... G7 (blue branches): conduction, indoor convection
G[4:8] = 1 / (w / λ + 1 / hi) * So

#### Interior walls: indoor convection, conduction indoor convection
The [equivalent conductance](https://en.m.wikipedia.org/wiki/Series_and_parallel_circuits#Combining_conductances) for the indoor walls is formed by the (yellow in Figures 2 and 3):
- convetion between wall and indoor air;
- conduction into the wall;
- convetion between wall and indoor air.

It is of the form:
$$G_k = \frac{1}{\frac{1}{h_i} + \frac{w}{\lambda} + \frac{1}{h_i}}S_{i,j}, \text{ with } k = 8 ... 12, \text{ and } j = 0 ... 4$$

where $S_{i,j}$ is the surface area of the wall in contact with the indoor air:
- $S_{i,0} = S_{i,1} = l \, H$;
- $S_{i,2} = S_{i,3} = S_{i,4} = L \, H$.

In [8]:
# G8 ... G12 (yellow branches): indoor walls
#    indoor convection, conduction, indoor convection
Si = np.array([l, l, L, L, L]) * H
G[8:13] = 1 / (1 / hi + w / λ + 1 / hi) * Si

#### Ventilation: heat transfer by advection

Ventilation of rooms 2 and 4 transfers heat to these spaces by [advection](https://en.m.wikipedia.org/wiki/Advection) (Figure 4). The [energy flow rate carried by the fluid](https://en.m.wikipedia.org/wiki/Mass_flow_rate) is:
$$\dot H = \dot m c \theta$$
where:
- $\dot H$ is the [enthalpy](https://en.m.wikipedia.org/wiki/Enthalpy#Relationship_to_heat) rate carried by the air, W;
- $\dot m$ - mass flow rate of the air, kg / s;
- $c$ - [specific heat capacity](https://en.m.wikipedia.org/wiki/Specific_heat_capacity) of the air, J / (kg⋅K);
- $\theta$ - air temperature, °C.

##### Air-flow: outdoor → room 2 → room 4 → outdoor

If the airflow is from outdoor to room 2, then the heat flow rate tranferred from outdoor air to the indoor air of room 2 is (Figure 4 a):
$$q_{13} = \dot m c T_o - \dot m c \theta_3$$
or
$$q_{13} = G_{13}(T_o - \theta_3)$$
where $G_{13} = \dot m c$.

The heat flow rate transferred by advection of the air from room 2 to room 4 is (Figure 4 a):
$$q_{14a} = \dot m c \theta_3 - \dot m c \theta_6$$
or
$$q_{14a} = G_{14}(\theta_3 - \theta_6)$$
where $G_{14} = \dot m c$.

![fig](./figures/08_vent_advection.svg)
> Figure 4. Heat advection by ventilation. a) Air-flow: outdoor  → room 2 → room 4 → outdoor. b) Air-flow: outdoor → room 4 → room 2 → outdoor.

##### Air-flow: outdoor → room 4 → room 2 → outdoor

If the airflow is from room 4 to room 2 (Figure 4 b), then the heat flow rate tranferred to the indoor air of room 2 is:
$$q_{14b} = \dot m c \theta_6 - \dot m c \theta_3$$
or
$$q_{14b} = G_{14}(\theta_6 - \theta_3)$$
where $G_{14} = \dot m c$.

The heat flow rate transferred by advection from the outdoor air to room 4 is:
$$q_{15} = \dot m c T_o - \dot m c \theta_6$$
or
$$q_{15} = G_{15}(T_o - \theta_6)$$
where $G_{14} = \dot m c.$

##### Heat flow rate from room 2 to room 4

If the air flows from room 2 to room 4, then the heat flow rate is $q_{14a} \equiv q_{14}.$ If the air flows from room 4 to room 2, then the heat flow rate is $q_{14b} \equiv -q_{14}.$ Therefore, a single heat flow rate, $q_{14}$, is needed in the thermal network (Figure 3).

##### Modelling the change in airflow direction

If the air flows in the direction: outdoor  → room 2 → room 4 → outdoor, then conductance $G_{15}$ is zero and conductances $G_{13}$ and $G_{14}$ are equal to $\dot m c$ (Figure 3 and Figure 4a).

If the air flows in the direction: outdoor  → room 2 → room 4 → outdoor, then conductance $G_{13}$ is zero and conductances $G_{14}$ and $G_{15}$ are equal to $\dot m c$ (Figure 3 and Figure 4b).

If there is no ventilation, the conductances $G_{13}$, $G_{14},$ and $G_{15}$ are zero.

In [9]:
# G13 ... G15 (green branches): advection by ventilation
G[13:16] = np.zeros(3)

> Table 1. Typical values for the ventilation rates (in air changes per hour, ACH) as a function of the position of windows (H. Recknagel, E. Spenger, E_R Schramek (2013), Table 1.12.1-4)

| Position of windows                     | Ventilation rate, ACH [h⁻ⁱ] |
| --------------------------------------- | ---------------------- |
| Window closed, doors closed             | 0 to 0.5 |
| Tilted window, venetian blind closed    | 0.3 to 1.5 |
| Tilted window, whitout venetian blind   | 0.8 to 4.0 |
| Window half opened                      | 5 to 10 |
| Window fully open                       | 9 to 15 |
| Window and French window fully open (cross ventilation) | about 40 |

#### Proportional controller

The heating, ventilation, and air conditioning ([HVAC](https://en.wikipedia.org/wiki/Heating,_ventilation,_and_air_conditioning)) system maintains the indoor air temperature at [setpoint](https://en.wikipedia.org/wiki/Setpoint_(control_system)) value. Therefore, the HVAC system may be modelled as a [proportional controller](https://en.m.wikipedia.org/wiki/Proportional_control) (Figure 5a) for wich the heat flow rate of the HVAC system is:

$$q = K_P (T_{sp} - \theta)$$

where:
- $K_P$ is the [proportional gain;](https://en.wikipedia.org/wiki/Proportional_control#Theory)
- $T_{sp}$ - setpoint temperature, °C;
- $\theta$ - measured temperature of the indoor air, °C.

The heat flow rate can be then expressed as (Figure 5b):
$$q = G (T_{sp} - \theta)$$
where $G \equiv K_p$.

The indoor air temperature control by the HVAC system (Figure 5a) may be modelled by a source of temperature, $T_{sp}$, and a conductance, $G$ (Figure 5b).

![fig](./figures/08_P-controller.svg)
> Figure 5. Model of a proportional controller in a thermal network (example for room 1). a) Principle of a temperature control system using a P-controller. b) Model of temperature control system in a thermal network.

If the rooms are in "free-runnning" (i.e., the controllers are not active), the proportional gains of the controllers are zero.

In [10]:
# G16 ... G19 (red branches): gains of proportional controllers
G[16:20] = np.zeros(4)

### Vector of temperature sources

Every edge (or branch) of the graph has a [temperature source](https://en.m.wikipedia.org/wiki/Voltage_source) which is not figured if it is [identically zero](https://en.wiktionary.org/wiki/identically_zero) (Figure 3a). The number of temperature sources is equal to the number of heat flow rates, i.e., the number of rows of the incidence matrix $A$.

In [11]:
# Vector of temperature sources
# =============================
b = np.zeros(A.shape[0])

The temperature sources modelling the outdoor temperature of the walls are on branches 0 ... 3.

In [12]:
b[0:4] = To         # cyan branches: outdoor temperature for walls

The temperature sources modelling the outdoor temperature for the ventilation are on braches 13 and 15.

In [13]:
b[[13, 15]] = To    # green branches: outdoor temperature for ventilation

The temperature sources modelling the [setpoint](https://en.m.wikipedia.org/wiki/Setpoint_(control_system)) temperatures are on the branches 16, ... 19. The setpoints for room 1 and 3 are 20 °C and 22 °C, respectively.  

In [14]:
b[[16, 18]] = 20, 22    # red branches: setpoints room 1 & 3

### Vector of heat flow rate sources

In our model, the heat flow rate sources are the solar radiation absorbed by the exterior walls. First, we will consider that the solar radiation is zero.

The number of heat flow rate sources is equal to the number of temperature nodes, i.e., the number of columns of the incidence matrix $A$.

In [15]:
# Vector of flow-rate sources
# =============================
f = np.zeros(A.shape[1])

### Outputs

In this problem, we are looking for the indoor air temperatures of the rooms (1, 2, 3, and 4) and for the thermal loads of the rooms.

The air temperatures of the rooms are $\theta_2$, $\theta_3$, $\theta_4$, and $\theta_6$,

The [thermal load](https://en.m.wikipedia.org/wiki/Cooling_load) is the heat flow rate needed to compensate all the other heat flow rates in order to maintain the air temperature at its setpoint. Therefore, the thermal loads are the heat flow rates of the controllers, i.e. $q_{16 ... 19}$.

In [16]:
# Indexes of outputs
# ==================
indoor_air = [2, 3, 4, 6]   # indoor air temperature nodes
controller = range(16, 20)  # controller branches

## Case 1: all 4 rooms are controlled

Considering that all four rooms are controlled by a HVAC system at the setpoint temperatures 20 °C, 20 °C, 22 °C, 18 °C for rooms 1, 2, 3, and 4, respectively and that the there is no solar radiation on the external walls, find the thermal loads of the four rooms.

For an almost perfect proportional controller, we will consider a very large gain of the controller as compared to the maximum value of the conductance matrix.

In [17]:
print(f"Maximum value of conductance: {max(G):.0f} W/K")

Maximum value of conductance: 1830 W/K


In [18]:
b[controller] = 20, 20, 22, 18  # °C setpoint temperature of the rooms
G[controller] = 1e9             # P-controller gain

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("1. All 4 rooms controlled")
print("θ:", θ[indoor_air], "°C")
print("q:", q[controller], "W")

1. All 4 rooms controlled
θ: [20. 20. 22. 18.] °C
q: [2008.4 1322.6 2346.4 4270. ] W


## Case 2: rooms 2 & 4 in free-running

Let's consider that:
- there is solar radiation absorbed by the exterior walls, i.e. $f_k = E \, S_{o,j}, \text{ with } k = 0, 1, 5, 7 \text{ and } j = 0...3;$
- as before, rooms 1 and 3 are controlled by a HVAC system, i.e. $K_{P,j} \rightarrow \infty, \text{ with } j=16, 18;$
- rooms 2 and 4 are in "[free-running](https://hal.science/hal-01815799/document)", i.e. $K_{P,j} = 0, \text{ with } j=17, 19.$ 

Find the thermal loads of rooms 1 and 3 and the temperatures of the indoor air of rooms 2 and 4.

In [19]:
# Zone 2 & 4 free-running; solar rad; without ventilation
G[[17, 19]] = 0     # controller gains for room 2 & 4

# Solar radiation
exterior_wall = [0, 1, 5, 7]
f[exterior_wall] = E * So

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("2. 2 & 4 free-run w/o ventilation")
print("θ:", θ[indoor_air], "°C")
print("q:", q[controller], "W")

2. 2 & 4 free-run w/o ventilation
θ: [20.  13.1 22.  11.7] °C
q: [1719.8    0.  2057.8    0. ] W


## Case 3: ventilation from outdoor to room 2

Room 2 in ventilated with an air flow rate of $ACH$ (ACH is the number of air-changes per hour). The outdoor air enters in room 2, then it moves to room 4 and, from here, it gets outdoors.

Find the thermal loads of rooms 1 and 3 and the temperatures of the indoor air of rooms 2 and 4.

To model this direction of air flow, the conductances for ventilation are:
$$G_{13} = G_{14} = \dot m c; G_{15} = 0 $$

In [20]:
# Zone 2 & 4 free-running; solar rad;
# Ventilation outdoor -> room 2 -> room 4 -> outdoor
ventilation = range(13, 16)
G[ventilation] = m_dot * c, m_dot * c, 0

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("3. 2 & 4 free-run, ventilation out -> 2 -> 4 -> out")
print("θ:", θ[indoor_air], "°C")
print("q:", q[controller], "W")

3. 2 & 4 free-run, ventilation out -> 2 -> 4 -> out
θ: [20.  11.7 22.  11.6] °C
q: [1761.1    0.  2099.     0. ] W


## Case 4: ventilation from room 4 to room 2

The air flow in room 2 is reversed: from outdoors, the air enters in room 4, then is moves to room 2, from which it gets outdoors. The air flow rate of room 2 is the same as before.

Find the thermal loads of rooms 1 and 3 and the temperatures of the indoor air of rooms 2 and 4.

To model this direction of air flow, the conductances for ventilation are:
$$G_{13} = 0; G_{14} = G_{15} = \dot m c $$

In [21]:
# Zone 2 & 4 free-running; solar rad;
# Ventilation outdoor -> room 4 -> room 2 -> outdoor
G[ventilation] = 0, m_dot * c, m_dot * c

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)
print("4. 2 & 4 free-run, ventilation out -> 4 -> 2 -> out")
print("θ:", θ[indoor_air], "°C")
print("q:", q[controller], "W")

4. 2 & 4 free-run, ventilation out -> 4 -> 2 -> out
θ: [20.  12.8 22.  11.3] °C
q: [1750.5    0.  2088.4    0. ] W


# Influence of physical parameters
## Influence of materials 

The materials used to constitute the walls is a first important parameter to ensure the thermal comfort of the users in a building, and to reduce the energy to use to control the temperature of the rooms. The main characteristic of a material that can influence the thermal behavior of a building is its conductivity: this can vary greatly from one material to another. Concrete, for example, has a conductivity of about 1.7 W/(mK), while wood is almost 10 times less conductive on average, with conductivities ranging from 0.1 W/(mK) to 0.2 W/(mK). [ref : http://www.ganeeva.fr/]

However, conductivity alone is not sufficient to compare two materials: it is the resistance, defined by equation [eq], and which therefore also depends on the wall thickness, that is of physical importance. Since wall thickness is largely determined by mechanical strength constraints, two materials cannot be properly compared using the same wall thickness value. 

In this part, we will compare 4 solutions using different materials: concrete walls, steel clad walls, wood clad walls and raw earth walls. These solutions will be compared in two different contexts : in winter and summer, by using the values associated with it in the introduction of the report.  Wall thicknesses typical of their practical use will be used. In the particular case of raw earth, the thickness used for the calculations corresponds to the required wall thickness to obtain the same mechanical resistance as a 15cm-wide regular concrete wall.  In this modelisation, all the rooms are controlled in temperature, according to the default parameters defined earlier. It should be noted that this simulation is only a model to compare approximately different types of facade materials. In practice, the walls are also insulated, which is why the values obtained are relatively high.
[ref conductivity concrete, wood and steel : https://fr.wikipedia.org/wiki/Liste_de_conductivit%C3%A9s_thermiques]

[ref conductivity raw earth : https://www.construction21.org]

The results are listed in the following tables : 

Winter : E = 200 W/m², T0 = 0°C : <br>
<img src="./figures/Materials_winter.png" alt="isolated" width="700"/>

Summer : E = 400 W/m², T0 = 30°C : <br>
<img src="./figures/Materials_summer.png" alt="isolated" width="700"/>



It can be seen in this table that the material of the walls is a determining factor in the energy to be used to maintain the building at temperature. Between the two extreme solutions: raw earth and steel, we observe a ratio of thermal flows higher than 7 in summer and winter. Overall, conventional materials such as steel and concrete have a rather poor thermal behavior, while wood and raw earth have good thermal properties. Even with a very small wall thickness, wood, for example, performs better than conventional materials. In practice, the worse the thermal material performance, the greater will be the thickness of the insulation.



## Influence of ventilation
The ventilation of a building is essential to ensure the comfort of its users, by controlling the air quality and humidity of the building. This phenomenon also has a significant influence on the thermal performance of the building and must therefore be taken into account in the calculations.

To study the influence of ventilation on the thermal management of the building, we will proceed by carrying out two types of modeling: the calculation of the thermal flows necessary to control the temperature of the building as a function of the ventilation rate, and the calculation of the interior temperatures (uncontrolled) in the presence of ventilation. 

- Influence of the ventilation rate on the thermal flows

For this calculation, we will take the default parameters defined in the introduction, in winter and in summer. We will calculate the control heat flows in each room, for ventilation rate values of 0.5 vol/h, 1 vol/h and 2 vol/h. In this modelisation, air is entering in the building from room 2 and going out outside in room 4. The results are presented in the following tables: 

Winter : E = 200 W/m², T0 = 0°C : <br>
<img src="./figures/Ventilation_winter.png" alt="isolated" width="700"/>

Summer : E = 400 W/m², T0 = 30°C : <br>
<img src="./figures/Ventilation_summer.png" alt="isolated" width="700"/>

We can therefore see that, in summer as in winter, the phenomenon of natural ventilation globally increases the energy to be used to maintain the temperature of the rooms, with heating in winter and with air conditioning in summer. Indeed, the outside temperature being colder than the control temperature in winter, and warmer in summer, the contribution of a mass of air inside the building leads to an additional heat flow by convection which is the opposite of that produced by the regulators. From a purely energy point of view, the ventilation of outside air represents an additional cost. Natural ventilation can therefore be interesting provided that the air used does not have a temperature difference that is too great in relation to the control temperatures of the rooms, which can be achieved using technologies such as the Canadian well.

- Influence of the ventilation rate on room temperatures without thermal control

Another way to study the influence of ventilation on building management is to calculate the equilibrium temperature of some rooms, not temperature controlled, for different values of ventilation rate. In the following, the calculations will be made in summer and winter, for ventilation rates of 0.5 vol/h, 1vol/h and 2vol/h.

If the ventilation rate is increased when the outdoor temperature is lower than the indoor temperature of the controlled rooms, then the temperature of the uncontrolled rooms decreases. This is logical because more fresh air is drawn into the uncontrolled rooms. If the ventilation rate is reduced, the opposite phenomenon occurs.


> Table 2 - Example : Room 2 in ventilated with an air-flow rate of 𝐴𝐶𝐻. The outdoor air enters in room 2, then it moves to room 4 and, from here, it gets outdoors. Find the temperature of the indoor air of **room 2**.

| Ventilation rate, ACH [h⁻ⁱ]         | Temperature [°C] |
| ------------------------------------| ---------------------- |
|0                                    | 13.1|
| 0.5                                 | 12.3 |
| 1                                   | 7.2 |
| 25                                  | 5.1 |



- Influence of the radiative forcing

>Table 3 : Winter conditions, $T_{ext}$=0°C. Temperature of the indoor air of **room 2**, as a function of the short-wave solar radiation absorbed E.

| E   [W/m²]     | Temperature [°C] |
| ------------------------------------| ---------------------- |
|0                                    | 8.2|
| 200                               | 13.1 |
| 400                               | 18|




>Table 4 : Summer conditions, $T_{ext}$=30°C. Temperature of the indoor air of **room 2**, as a function of the short-wave solar radiation absorbed E.

| E   [W/m²]     | Temperature [°C] |
| ------------------------------------| ---------------------- |
|0                                    | 26.5|
| 200                               | 31.4 |
| 400                               | 36.2|


## Influence of the adding of a solar protection

For better comfort in summer, it can be interesting to add a solar protection such as a cap which reduces the surface receiving solar radiation. This horizontal protection prevents the sun's rays from hitting all the building's facades in summer because the sun is high in the sky. This reduces the temperature inside unregulated rooms or reduces the energy input to regulate the rooms. In winter the sun's rays are not blocked because the sun is lower (see diagram below). This does not reduce the indoor temperature in winter.

<img src="./figures/Solar_cap.png" alt="isolated" width="700"/>


>Table 5 : Summer conditions, $T_{ext}=30°C$ and $E=400W/m²$ - Rooms 1 and 3 are controlled by HVAC, Room 2 and 4 are free-running.
Temperature of the indoor air of **room 2**.

| Percentage of S0  [%]   | Indoor temperature [°C] |
| ------------------------------------| ---------------------- |
|100                                   | 36.2|
|75                            | 34.6 |
|50                              | 32.3|
|25                             | 28.5|

>Table 6 : Summer conditions, $T_{ext}=30°C$ and $E=400W/m^2$ - All rooms are controlled by a HVAC system at the setpoint :
$T_1=20°C$, $T_2=20°C$, $T_3=22°C$, $T_4=18°C$.
Thermal load of **room 1** :

| Percentage of S0  [%]   | q [W] |
| ------------------------------------| ---------------------- |
|100                                   | 857|
|75                            | 618|
|50                              | 380|
|25                             | 141|

# Statical model for a complex architecture

In this part, we focus on the modeling of a more complex architecture, resulting from the TET option project "Ferme du Milieu à Lhuis". This project aims to analyse the renovation of a farm in the department of the Ain with the aim of transforming it into an associative and get-together place. 

The main objective of this part is to set up a statical thermal model, with Python, of the ground floor of the farm once renovated. The work is based on Christian Ghiaus' course which models a 5-room building.

## Problem
### Data

The future geometries of the ground floor of the farm are described in a plan provided by the ENSAL students (architecture school) with whom we work in collaboration.

<img src="./figures/bldg_plan.png" alt="isolated" width="650"/>

> Figure 1. Ground floor plan


The thermal conductivities of the walls are: 
- $\lambda_1 = 0.052 \, \mathrm{W/(m \, K)}$ for the compressed straw.
- $\lambda_2 = 0.03 \, \mathrm{W/(m \, K)}$ for internal walls.
- $\lambda_3 = 0.87 \, \mathrm{W/(m \, K)}$ for cinder blocks.

The convective heat coefficients between the walls and the indoor and outdoor air are $h_i = 8 \, \mathrm{W/(m^2 \, K)}$ and $h_o = 25 \, \mathrm{W/(m^2 \, K)}$ respectively.

The annual mean short-wave solar radiation absorbed by each wall is $E = 165.33 \, \mathrm{W/m^2}$.

The annual mean outdoor temperature is $T_o = 12,3 \, \mathrm{°C}$.


In [22]:
import numpy as np

np.set_printoptions(precision=1)

#%% Data

#set temperatures (°C)

T1=20
T2=20
T3=20
T4=20
T5=20
T6=20

# outdoor temperature
T0 = 12.3           # °C

# short-wave solar radiation absorbed by each wall
E = 165.33          # W/m2

# thermo-physical propertites
λ1 = 0.052           # W/(m K) wall thermal conductivity
λ2 = 0.03 
λ3 = 0.87 
hi, ho = 8, 25      # W/(m2 K) convection coefficients in, out

## Thermal model

### Thermal conductances

In the model, we will consider the conductances:
- $h \, S$ - convection, W/K;
- $\lambda / w \, S$ - conduction, W/K;
- $K_p$ - gain of the proportional controller, W/K.

**Note:** in determening the surfce area of the walls, the lengths of the walls are the indoor values.

We will consider the conductances of exterior and the interior walls (Figure 2). The exterior walls are modelled with a conductance for outdoor convection and an equivalent conductance for conduction in the wall and indoor convection. Blue walls are composed of two layers of cinder blocks and compressed straw, green walls are only made out of cinder blocks.
The indoor walls are modelled by an equivalent conductance for indoor convection, conduction, and indoor convection on the other side.

<img src="./figures/bldg_plan2.png" alt="isolated" width="650"/>
> Figure 2. Walls forming thermal coductances in the model.

In [23]:
# dimensions (m)
H=3

L1=9.45 
L2=4.9
L3=L2
L4=6.79
L5=1.55
L6=16.19
L7=7.71 
L8=4.07
L9=4.27
L10=24.6
L11=12.52
L12=3.01

w1=0.3
w2=0.1
w3=0.2

### Thermal network

The thermal network modeling the building is a [weighted directed graph](https://en.m.wikipedia.org/wiki/Directed_graph) (Figure 3). It is composed of edges (or branches) and vertexes (or nodes).

The temperature nodes model a volume, a surface, a line or a point in space for which the temperature is considered homogenous. The nodes are dependent temperatures (outputs of the physical system). The reference temperature is considered zero Celsius; it has a known value and is represented by a bar (Figure 3).

To a temperature node $\theta_j$, there are connected [heat flow rate](https://en.m.wikipedia.org/wiki/Rate_of_heat_flow) branches $q_k, k = 0 ... n$ and [heat flow rate sources](https://en.m.wikipedia.org/wiki/Current_source), $f_j$ (Figure 3a). A heat rate source delivers a heat flow rate whatever is the temperature difference. In buildings, typical heat flow rate sources are [solar irradiance](https://en.m.wikipedia.org/wiki/Solar_irradiance), [electrical appliances](https://en.wikipedia.org/wiki/Joule_heating) and [occupants](https://en.wikipedia.org/wiki/Thermal_comfort#Metabolic_rate). If a heat flow rate source is [identically zero](https://en.wiktionary.org/wiki/identically_zero), then it is not figured on the graph.

In steady-state, the [algebraic sum of the heat flow rates](https://en.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws#Kirchhoff's_current_law) entering the temperature node $\theta_j$ is zero:
$$\sum_{k} q_k + f_j= 0$$

Two temperature nodes $\theta_i$ and $\theta_j$ are connected by directed edges of heat flow rates $q_k$ (Figure 3a). On each edge, there is a conductance $G_k$ and a temperature source $b_k$. A [temperature source](https://en.m.wikipedia.org/wiki/Voltage_source) maintains a temperature value whatever is the heat flow rate. In building, outdoor air, ground, and setpoint temperatures are typical examples of temperature sources. If a temperature source is [identically zero](https://en.wiktionary.org/wiki/identically_zero), then it is not figured on the graph.

From the [directed sum of temperature differences over a branch](https://en.m.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws#Kirchhoff's_voltage_law), it results that the [temperature drop](https://en.wikipedia.org/wiki/Voltage_drop) over a conductance is (Figure 3a):
$$e_k = \theta_i - \theta_j + b_k$$

The relation between the temperature drop, $e_k$, over a conductance, $G_k$, and the heat flow rate, $q_k$, thrrough the conductance $G_k$ is:
$$q_k = G_k \, e_k$$

In figure 3b, the heat flow rate branches $q_k$ are not represented. They are in fact going through each corresponding $G_k$ looking from left to right or downward.

This [weighted directed graph](https://en.m.wikipedia.org/wiki/Directed_graph) is characterized by:
- $A$ - [incidence matrix](https://en.m.wikipedia.org/wiki/Incidence_matrix);
- $G$ - [conductance](https://en.m.wikipedia.org/wiki/Electrical_resistance_and_conductance)  diagonal matrix;
- $b$ - vector of [temperature sources](https://en.m.wikipedia.org/wiki/Voltage_source);
- $f$ - vector of [flow-rate sources](https://en.m.wikipedia.org/wiki/Current_source).


The solution of the problem is ([Ghiaus (2013)](https://hal.archives-ouvertes.fr/hal-03605823/document)):

$$ \theta = (A^TGA)^{-1}(A^TGb + f)$$
and
$$ q = G(-A\theta + b)$$

<img src="./figures/thermalmodelexample.png" alt="isolated" width="200"/>

> Figure 3a. Thermal network: thermal network model for the buiding shown in Figures 1 and 2.

<img src="./figures/RCmodel.svg" alt="isolated" width="650"/>

> Figure 3b. Thermal network: thermal network model for the buiding shown in Figures 1 and 2.

For our model, the graph has 27 braches and 12 nodes (Figure 3b).

In [24]:
nq, nθ = 27, 12  # number of flow-rates branches and of temperaure nodes

### Incidence matrix
The [incidence matrix of a directed graph](https://en.m.wikipedia.org/wiki/Incidence_matrix) shows the relationship between the edges (or branches) $k$ and the vertexes (or nodes) $j$ of the graph (Figure 3a):

$$A_{k,j} = \begin{cases}\phantom{-}
0 & \text{if heat flow rate } q_k \text{ is not connected to node }  \theta_j \\ 
+1 & \text{if heat flow rate } q_k \text{ enters into node }  \theta_j\\ 
-1 & \text{if heat flow rate } q_k \text{ gets out of node }  \theta_l 
\end{cases}$$

The incidence matrix is a [sparce matrix](https://en.m.wikipedia.org/wiki/Sparse_matrix) which has:
- the number of rows equal to $n_q$, the number of flow rates $q_k$;
- the number of columns equal to $n_{\theta}$, the number of temperature nodes $\theta_j$.

In [25]:
#%% Incidence matrix

A = np.zeros([nq, nθ])

# q0 ... q5 (blue branches)
A[0, 0] = 1
A[1, 1] = -1
A[2, 2] = 1
A[3, 3] = 1
A[4, 4] = -1
A[5, 5] = -1


# q6 ... q11 (violet branches)

A[6, 0], A[6, 7] = -1, 1
A[7, 1], A[7, 8] = -1, 1
A[8, 2], A[8, 6] = -1, 1
A[9, 3], A[9, 9] = -1, 1
A[10, 10], A[10, 4] = -1, 1
A[11, 11], A[11, 5] = -1, 1


# q12 ... q20 (orange branches)

A[12, 6], A[12, 7] = -1, 1
A[13, 7], A[13, 8] = -1, 1
A[14, 6], A[14, 9] = -1, 1
A[15, 6], A[15, 10] = -1, 1
A[16, 7], A[16, 10] = -1, 1
A[17, 8], A[17, 10] = -1, 1
A[18, 9], A[18, 10] = -1, 1
A[19, 8], A[19, 11] = -1, 1
A[20, 10], A[20, 11] = -1, 1



# q21 ... q26 (red branches)

A[21, 7] = 1
A[22, 6] = 1
A[23, 8] = -1
A[24, 9] = 1
A[25, 10] = -1
A[26, 11] = -1

### Conductances
The number of conductances corresponds to the number of flows which is the number of rows of incidence matrix $A$

In [26]:
# Conductance matrix
# ==================
G = np.zeros(A.shape[0])

#### Exterior walls: outdoor convection
The conductances of [convection](https://en.m.wikipedia.org/wiki/Newton%27s_law_of_cooling) between the walls and the outdoor air are of the form
$$G_i = h_o \, S$$
where $S$ is the surface area of the wall in contact with the outdoor air: for example $S= L_2 \, H$ for the room 2 in Figure 2


In [27]:
# G0 ... G5 : outdoor convection (blue)

S0=L2*H
G[0]= ho*S0

S1=(L3+L4)*H
G[1]= ho*S1

S2=(L1+L9)*H
G[2]= ho*S2

S3=(L8+L7-L12-w3)*H
G[3]= ho*S3

S4=L5*H
G[4]= ho*S4

S5=(2*L11+L10+L10-L12-w2-L6-w1-w3)*H
G[5]= ho*S5

#### Exterior walls: conduction and indoor convection
The [equivalent conductance](https://en.m.wikipedia.org/wiki/Series_and_parallel_circuits#Combining_conductances) for [conduction](https://en.m.wikipedia.org/wiki/Thermal_conduction) in the wall and [convection](https://en.m.wikipedia.org/wiki/Newton%27s_law_of_cooling) between the wall and the indoor air (example for the blue walls in Figure 2 which are made of cinder bkocks and compressed straw) is:
$$G_k = \frac{1}{\frac{w_2}{\lambda_2} + \frac{w_3}{\lambda_3} + \frac{1}{h_i}}S$$ 

In [28]:
# G6 ... G11 conduction + indoor convection (violet)

hbleu=1/(1/hi+w2/λ2+w3/λ3)

hvert=1/(1/hi+w3/λ3)

G[6]= hbleu*S0

G[7]= hbleu*S1

G[8]= hbleu*S2

G[9]= hbleu*S3

G[10]= hbleu*S4

G[11]= hvert*S5

#### Interior walls: indoor convection, conduction indoor convection
The [equivalent conductance](https://en.m.wikipedia.org/wiki/Series_and_parallel_circuits#Combining_conductances) for the indoor walls is formed by the (yellow in Figures 2 and 3):
- convetion between wall and indoor air;
- conduction into the wall;
- convetion between wall and indoor air.

It is of the form:
$$G_k = \frac{1}{\frac{1}{h_i} + \frac{w}{\lambda} + \frac{1}{h_i}}S$$

where $S$ is the surface area of the wall in contact with the indoor air.

In [29]:

# G12 ... G20 indoor walls (orange)
# indoor convection + conduction + indoor convection 

hjaune=1/(2/hi+w2/λ2)

hbleuvert=1/(2/hi+w1/λ1+w3/λ3)

S12=L9*H
G[12]=hjaune*S12

G[13]=hjaune*S12

S14=L7*H
G[14]=hjaune*S14

S15=(L1-L7-w2)*H
G[15]=hjaune*S15

G[16]=hjaune*S0

S17=((L8-L5-w2)+L3)*H
G[17]=hjaune*S17

S18=L8*H
G[18]=hjaune*S18

S19=L12*H
G[19]=hbleuvert*S19

S20=L6*H
G[20]=hbleuvert*S20

#### Proportional controller

The heating and air conditioning ([HVAC](https://en.wikipedia.org/wiki/Heating,_ventilation,_and_air_conditioning)) system maintains the indoor air temperature at [setpoint](https://en.wikipedia.org/wiki/Setpoint_(control_system)) value. Therefore, the HVAC system may be modelled as a [proportional controller](https://en.m.wikipedia.org/wiki/Proportional_control) (Figure 5a) for wich the heat flow rate of the HVAC system is:

$$q = K_P (T_{sp} - \theta)$$

where:
- $K_P$ is the [proportional gain;](https://en.wikipedia.org/wiki/Proportional_control#Theory)
- $T_{sp}$ - setpoint temperature, °C;
- $\theta$ - measured temperature of the indoor air, °C.

The heat flow rate can be then expressed as (Figure 5b):
$$q = G (T_{sp} - \theta)$$
where $G \equiv K_p$.

The indoor air temperature control by the HVAC system (Figure 5a) may be modelled by a source of temperature, $T_{sp}$, and a conductance, $G$ (Figure 5b).

![fig](./figures/08_P-controller.svg)
> Figure 5. Model of a proportional controller in a thermal network (example for room 1). a) Principle of a temperature control system using a P-controller. b) Model of temperature control system in a thermal network.

If the rooms are in "free-runnning" (i.e., the controllers are not active), the proportional gains of the controllers are zero. Here, all the rooms are controlled except the 6th.

In [30]:
# G21 ... G26 gains of proportional controllers (red)

K0=0
Kinf=1e9

#Room 1

G[22]=Kinf

#Room 2

G[21]=Kinf

#Room 3

G[23]=Kinf

#Room 4

G[24]=Kinf

#Room 5

G[25]=Kinf

#Room 6

G[26]=K0

### Vector of temperature sources

Every edge (or branch) of the graph has a [temperature source](https://en.m.wikipedia.org/wiki/Voltage_source) which is not figured if it is [identically zero](https://en.wiktionary.org/wiki/identically_zero) (Figure 3a). The number of temperature sources is equal to the number of heat flow rates, i.e., the number of rows of the incidence matrix $A$.

In [31]:
# Vector of temperature sources
# =============================
b = np.zeros(A.shape[0])

The temperature sources modelling the outdoor temperature of the walls are on branches 0, ... 5.

In [32]:
#blue branches : outdoor temperature for walls
b[0:6]=T0,-T0,T0,T0,-T0,-T0

The temperature sources modelling the [setpoint](https://en.m.wikipedia.org/wiki/Setpoint_(control_system)) temperatures are on the branches 21, ... 26. The setpoints for all rooms is 20 °C.  

In [33]:
#red branches : setpoints rooms 
b[21:27]=T2,T1,-T3,T4,-T5,-T6

### Vector of heat flow rate sources

In our model, the heat flow rate sources are the solar radiation absorbed by the exterior walls. First, we will consider that the solar radiation is zero.

The number of heat flow rate sources is equal to the number of temperature nodes, i.e., the number of columns of the incidence matrix $A$.

In [34]:
# Vector of flow-rate sources
# =============================
f = np.zeros(A.shape[1])

The solar radiation $E$ is represented Figure 3b by $Q_{1...6}$. The solar radiation absorbed by the exterior walls is 

$Q=E*S$.

In [35]:
#Solar radiation

f[0]=E*S0

f[1]=E*S1

f[2]=E*S2

f[3]=E*S3

f[4]=E*S4

f[5]=E*S5

## Results

In this problem, we are looking for the indoor air temperatures of the rooms (1, 2, 3, and 4) and for the thermal loads of the rooms.

The air temperatures of the rooms are nods 6
...11.

The [thermal load](https://en.m.wikipedia.org/wiki/Cooling_load) is the heat flow rate needed to compensate all the other heat flow rates in order to maintain the air temperature at its setpoint. Therefore, the thermal loads are the heat flow rates of the controllers, i.e. $q_{21 ... 26}$.

In [36]:
#%%Solving

θ = np.linalg.inv(A.T @ np.diag(G) @ A) @ (A.T @ np.diag(G) @ b + f)
q = np.diag(G) @ (-A @ θ + b)


q_affich = np.zeros(6) #variable qui remet dans l'ordre les flux pour mieux afficher les données
q_affich[0:7]=[q[22], q[21], -q[23], q[24], -q[25], -q[26]]

print("Room temperatures", θ[6:12], "°C")
print("Thermal loads", q_affich, "W")
print("Total consumption : ", round(sum(q_affich),2) ,"W")

Room temperatures [20.  20.  20.  20.  20.  18.9] °C
Thermal loads [12.   4.3 11.8  7.5  9.6 -0. ] W
Total consumption :  45.16 W


# Conclusion
This work allowed us to understand the influence of different parameters in the behavior of a simple building. We also studied the modeling of a more complex architecture, resulting from the TET option project "Ferme du Milieu à Lhuis".


## References

1. C. Ghiaus (2013) Causality issue in the heat balance method for calculating the design heating and cooling loads, *Energy* 50: 292-301, https://doi.org/10.1016/j.energy.2012.10.024, open access preprint: [hal-03605823](https://hal.archives-ouvertes.fr/hal-03605823/document)

2. H. Recknagel, E. Sprenger, E.-R. Schramek (2013) Génie climatique, 5e edition, Dunod, Paris. ISBN 978-2-10-070451-4

3. http://www.ganeeva.fr

4. Ross, M., Korany, Y. and Lohonyai, A. (2011) Application of Poly-Mor Foam as Insulation for Single Wythe Masonry Walls, Table 8 -Thermal Resistance of Various Insulating Materials. Available at: https://doi.org/10.7939/R3J67905P.

5. JRC Photovoltaic Geographical Information System (PVGIS) - European Commission. Available at: https://re.jrc.ec.europa.eu/pvg_tools/fr/tools.html


